# Creating HF Dataset for Mistral Fine-tuning

Code authored by: Shaw Talebi <br>
Video link: https://youtu.be/XpoKB3usmKc <br>
Blog link: https://medium.com/towards-data-science/qlora-how-to-fine-tune-an-llm-on-a-single-gpu-4e44d6b5be32 <br>
<br>
Colab link: https://colab.research.google.com/drive/1AErkPgDderPW0dgE230OOjEysd0QV1sR?usp=sharing <br>
Dataset link: https://huggingface.co/datasets/shawhin/shawgpt-youtube-comments <br>
Model link: https://huggingface.co/shawhin/shawgpt-ft

In [3]:
import csv
import random
from datasets import Dataset, DatasetDict

### prep training examples

In [5]:
# load csv of YouTube comments
comment_list = []
response_list = []

with open('data/YT-comments.csv', mode ='r') as file:
    file = csv.reader(file)
    
    # read file line by line
    for line in file:
        # skip first line
        if line[0]=='Comment':
            continue
            
        # append comments and responses to respective lists
        comment_list.append(line[0])
        response_list.append(line[1] + " -ShawGPT")

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 3959: character maps to <undefined>

In [ ]:
# attempt 1 at prompt format
# intstructions_string = f"""ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
# It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
# ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
# thus keeping the interaction natural and engaging.
# """

# example_template = lambda comment, response: f'''<s>[INST] {intstructions_string} \nViewer: {comment} \nShawGPT: [/INST]''' + response + "</s>"

# example_list = []
# for i in range(len(comment_list)):
#     example = example_template(comment_list[i],response_list[i])
#     example_list.append(example)

# print(example_list[10])

In [7]:
# attempt 2 at prompt format
intstructions_string = f"""ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""

example_template = lambda comment, response: f'''<s>[INST] {intstructions_string} \n{comment} \n[/INST]\n''' + response + "</s>"

example_list = []
for i in range(len(comment_list)):
    example = example_template(comment_list[i],response_list[i])
    example_list.append(example)

print(example_list[10])

IndexError: list index out of range

In [6]:
# create train/test split
test_index_list = random.sample(range(0, len(example_list)-1), 9)

test_list = [example_list[index] for index in test_index_list]

for example in test_list:
    example_list.remove(example)

NameError: name 'example_list' is not defined

### create HF dataest

In [ ]:
data = DatasetDict({'train':Dataset.from_dict({"example":example_list}), 'test':Dataset.from_dict({"example":test_list})})

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 50
    })
    test: Dataset({
        features: ['example'],
        num_rows: 9
    })
})

### push dataset to hub

In [ ]:
# option 1: notebook login
from huggingface_hub import notebook_login
notebook_login()

# # option 2: key login
# from huggingface_hub import login
# write_key = 'hf_' # paste token here
# login(write_key)

In [ ]:
# push dataset to hub
data.push_to_hub("shawhin/shawgpt-youtube-comments")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/531 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/shawhin/shawgpt-youtube-comments/commit/eb6e890103c25bb7f4be2d8ce541dd2b320d46f9', commit_message='Upload dataset', commit_description='', oid='eb6e890103c25bb7f4be2d8ce541dd2b320d46f9', pr_url=None, pr_revision=None, pr_num=None)